In [1]:
import pandas as pd
import numpy as np
import Orange
import pickle
from sklearn.model_selection import StratifiedKFold #Ayudara a dividir los dataframes k veces para la validacion cruzada

### Cargue de los datasets

In [2]:
tic_tac_toe_data =Orange.data.Table("Datasets/Tic-Tac-Toe Endgame Data Set/Tic-Tac-Toe Endgame Data Set.xlsx")
credit_approval_data = Orange.data.Table("Datasets/Credit Approval Data Set/Credit Approval Data Set.xlsx")
wine_data = Orange.data.Table("Datasets/Wine Dataset/Wine Dataset.xlsx")

### Importando modelos arbol J48 creados en orange

In [3]:
tic_tac_toe_model_url="Datasets/Tic-Tac-Toe Endgame Data Set/Tree.pkcls"
credit_approval_model_url="Datasets/Credit Approval Data Set/Tree.pkcls"
wine_model_url="Datasets/Wine Dataset/Tree.pkcls"

with open(tic_tac_toe_model_url, "rb") as model1:
    tic_tac_toe_tree = pickle.load(model1)
    
with open(credit_approval_model_url, "rb") as model2:
    credit_approval_tree = pickle.load(model2)
    
with open(wine_model_url, "rb") as model3:
    wine_tree = pickle.load(model3)

In [4]:
tic_tac_toe_tree

TreeModel(data=<?>, root=<Orange.tree.DiscreteNode at 0x2354f1ddfd0>)

In [5]:
credit_approval_tree

TreeModel(data=<?>, root=<Orange.tree.DiscreteNode at 0x2354f224bb0>)

In [6]:
wine_tree

TreeModel(data=<?>, root=<Orange.tree.NumericNode at 0x2354f22dd90>)

### Se crean las funciones para hacer prueba por reescritura, leave one out y cross validation

Funcion para reescritura

In [7]:
def reescritura(datos, arbol, class_column_name):
    original_target = datos.get_column_view(class_column_name)[0].astype('int').tolist()
    predicciones = arbol(datos).tolist()
    #print('________________________________')

    #print('Original: '+str(original_target))
    #print('Predicciones: '+str(predicciones))
    
    #print('_______________________________')
    good_predictions=0
    for t,p in zip(original_target, predicciones):
        if t == p:
           good_predictions=good_predictions+1
    
    exactitud = good_predictions/len(original_target)
    return  exactitud

Funciones para validacion cruzada

In [8]:
def promedio(lst):
    return sum(lst) / len(lst)

In [9]:
def sub_arbol(data, class_column_name, train_index, test_index):
    train = data[min(train_index):max(train_index)]
    test = data[min(test_index):max(test_index)]
    tr = Orange.classification.TreeLearner()
    arbol = tr(train)
    exactitud = reescritura(test, arbol, class_column_name)
    return exactitud

In [10]:
def cross_validation(data, class_column_name, ndivisiones=5):
    X=pd.DataFrame(np.zeros((len(data), 1)))
    y=pd.DataFrame(np.zeros((len(data), 1)))
    skf = StratifiedKFold(n_splits=ndivisiones)
    lista_exactitud = []
    for k, (train_index, test_index) in enumerate(skf.split(X, y)):
        train_index = train_index.tolist()
        test_index=test_index.tolist()
        #print(len(train_index))
        #print(len(test_index))
        exactitud = sub_arbol(data, class_column_name, train_index, test_index)
        lista_exactitud.append(exactitud)
    return lista_exactitud

Funcion para leave one out

In [11]:
'''def leave_one_out(datos,class_column_name):
    domain = datos.domain
    for d in range(0,len(datos)):
        
        if d==0:
            test = datos[d]            
            train = datos[d+1:]            
        elif d == len(datos)-1:
            test = datos[d]
            train = datos[:d-1]
        else:
            test = datos[d]
            train1 = datos[:d]
            train2 = datos[d+1:]
            train=train1.concatenate(train2,axis=0)
            print(train)
    #return '''

'def leave_one_out(datos,class_column_name):\n    domain = datos.domain\n    for d in range(0,len(datos)):\n        \n        if d==0:\n            test = datos[d]            \n            train = datos[d+1:]            \n        elif d == len(datos)-1:\n            test = datos[d]\n            train = datos[:d-1]\n        else:\n            test = datos[d]\n            train1 = datos[:d]\n            train2 = datos[d+1:]\n            train=train1.concatenate(train2,axis=0)\n            print(train)\n    #return '

In [12]:
def leave_one_out(datos):
    leave_one_out=Orange.evaluation.testing.LeaveOneOut()
    learner=Orange.classification.TreeLearner()
    test=leave_one_out(datos, learners=[learner])
    exactitud=Orange.evaluation.CA()
    return exactitud(test)[0]

#### Analisis para Tic-Tac-Toe Endgame Data Set (Datos Simbolicos)

This database encodes the complete set of possible board configurations at the end of tic-tac-toe games, where "x" is assumed to have played first. The target concept is "win for x" (i.e., true when "x" has one of 8 possible ways to create a "three-in-a-row").

Reescritura

In [13]:
exactitud1 = reescritura(tic_tac_toe_data, tic_tac_toe_tree, 'Class')

In [14]:
print(f'En reescritura la exactitud es {exactitud1}')

En reescritura la exactitud es 0.941544885177453


Validacion cruzada

In [15]:
lista_exactitud_tic_tac_toe=cross_validation(tic_tac_toe_data, 'Class', 5)

In [16]:
print(lista_exactitud_tic_tac_toe)
print('En validacion cruzada la exactitud es',promedio(lista_exactitud_tic_tac_toe))

[0.4397905759162304, 1.0, 1.0, 1.0, 0.3157894736842105]
En validacion cruzada la exactitud es 0.7511160099200882


Leave One Out

In [17]:
exactitud_tic_tac_toe_loo=leave_one_out(tic_tac_toe_data)
print('En validacion leave one out la exactitud es',exactitud_tic_tac_toe_loo)

En validacion leave one out la exactitud es 0.8256784968684759


* Se puede observar que en reescritura el desempeño es mejor, pues la validacion cruzada y leave one out (que es una validacion cruzada solo dejando una instancia afuera) arroja esos resultados debido a que al hacer el test al modelo se le presentan datos nuevos, y son metodos muy buenos de prueba ya que se asemejan al comportamiento cuando el modelo se pone a prueba en un contexto real. 

#### Analisis para Credit Approval Data Set (Dataset con missing values)

This file concerns credit card applications.  All attribute names and values have been changed to meaningless symbols to protect confidentiality of the data.

Reescritura

In [18]:
exactitud2 = reescritura(credit_approval_data, credit_approval_tree, 'Class')

In [19]:
print(f'En reescritura la exactitud es {exactitud2}')

En reescritura la exactitud es 0.936231884057971


Validacion Cruzada

In [20]:
lista_exactitud_credit_approval=cross_validation(credit_approval_data, 'Class', 5)

In [21]:
print(lista_exactitud_credit_approval)
print('En validacion cruzada la exactitud es',promedio(lista_exactitud_credit_approval))

[0.6423357664233577, 0.9708029197080292, 0.9708029197080292, 0.9781021897810219, 0.8467153284671532]
En validacion cruzada la exactitud es 0.8817518248175183


Leave One Out

In [22]:
exactitud_credit_approval_loo=leave_one_out(credit_approval_data)
print('En validacion leave one out la exactitud es',exactitud_credit_approval_loo)

En validacion leave one out la exactitud es 0.8304347826086956


* Similar al caso anterior, reescritura gana en exactitud, pues se prueba el modelo con los mismos datos de entrenamiento. Con esto, sin embargo para este caso la validacion cruzada estuvo mejor que leave one out a comparacion del caso anterior, por lo que parece que posiblemente los missing values del dataset afecten en mayor medida a esta ultima validacion 

#### Analisis para wine dataset

These data are the results of a chemical analysis of wines grown in the same region in Italy but derived from three different cultivars. The analysis determined the quantities of 13 constituents found in each of the three types of wines.

Reescritura

In [23]:
exactitud3 = reescritura(wine_data,wine_tree,'target')

In [24]:
print(f'En reescritura la exactitud es {exactitud3}')

En reescritura la exactitud es 0.9887640449438202


Validacion Cruzada

In [25]:
lista_exactitud_wine=cross_validation(wine_data, 'target', 5)

In [26]:
print(lista_exactitud_wine)
print('En validacion cruzada la exactitud es',promedio(lista_exactitud_wine))

[0.9142857142857143, 0.9428571428571428, 1.0, 1.0, 0.7058823529411765]
En validacion cruzada la exactitud es 0.9126050420168067


Leave one out

In [27]:
exactitud_wine_loo=leave_one_out(wine_data)
print('En validacion leave one out la exactitud es',exactitud_wine_loo)

En validacion leave one out la exactitud es 0.9382022471910112


* En cuanto a este dataset, tampoco tiene missing values y leave one out estuvo mejor que validacion cruzada, sin embargo hay que tener en cuenta que para estos casos solo se evaluo para una validacion cruzada cuando el dataset es dividido 5 veces, con lo cual, puede ser posible que con una cantidad diferente de divisiones la exactitud mejore. <br><br>Ahora, teniendo en cuenta la exactitud que se obtuvo en reescritura con el arbol KD del ejercicio anterior, la cual fue de 0.9943820224719101, podemos ver que esta fue mejor que la prueba de reescritura usando J48, sin embargo, la cantidad y la complejidad de las reglas disminuye para este ultimo.

In [28]:
print(wine_tree.print_tree())

       [ 0. 14. 48.] Flavanoids ≤ 1.57
       [ 0. 13.  0.]     Color intensity ≤ 3.8
       [ 0.  1. 48.]     Color intensity > 3.8
       [59. 57.  0.] Flavanoids > 1.57
       [ 1. 53.  0.]     Proline ≤ 714
       [58.  4.  0.]     Proline > 714
          [0. 4. 0.]         Color intensity ≤ 3.4
       [58.  0.  0.]         Color intensity > 3.4

